In [1]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import h5py
import pickle
import pandas
import matplotlib.pyplot as plt
#import deepdish.io as io
import tensorflow as tf
from keras.models import Model,Sequential
from keras.layers import Input, Dense, Dropout
from keras.utils import plot_model
from keras.models import load_model
from sklearn.preprocessing import scale, normalize
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
from collections import Counter
import sys

Using TensorFlow backend.


In [2]:
encoder = load_model('/home/zp/gupta/jetAnomaly/utils/VAE-FCN-model-d80-UdeMdata-cycannealing-eps1_encoder.h5')

Instructions for updating:
Colocations handled automatically by placer.


/lcg/storage15/software64/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
f_encoded_train=h5py.File('/lcg/storage13/atlas/gupta/encoded_x_train_mid.h5','r')
f_encoded_test = h5py.File('/lcg/storage13/atlas/gupta/encoded_x_test_mid.h5','r')

In [11]:
encoded_x_train_mid=f_encoded_train['table']
encoded_x_test_mid=f_encoded_test['table']

In [12]:
level = 3
encoded_x_train = encoded_x_train_mid[-level]
encoded_x_test = encoded_x_test_mid[-level]

In [13]:
hash_size = 10
sample_size, vec_size = encoded_x_train.shape
projections = np.random.randn(hash_size, vec_size)
hashed_encoded = list()
for i in range(sample_size):
    bools = ''
    for j in range(hash_size):
        bool = (np.dot(encoded_x_train[i], projections[j].T) > 0).astype('int')
        #print(bool)
        bools += str(bool)
        #print(bools)
    hashed_encoded.append(bools)
len(hashed_encoded)

493900

In [14]:
def getDuplicatesWithInfo(hashed_encoded):
    ''' Get duplicate element in a list along with thier indices in list
     and frequency count'''
    dictOfElems = dict()
    index = 0
    # Iterate over each element in list and keep track of index
    for elem in hashed_encoded:
        # If element exists in dict then keep its index in lisr & increment its frequency
        if elem in dictOfElems:
            dictOfElems[elem][0] += 1
            dictOfElems[elem][1].append(index)
        else:
            # Add a new entry in dictionary 
            dictOfElems[elem] = [1, [index]]
            #dictOfElems[elem] = 1
        index += 1    
 
    dictOfElems = { key:value for key, value in dictOfElems.items() }
    return dictOfElems

In [15]:
dictOfElems = getDuplicatesWithInfo(hashed_encoded)
listHash = list()
listOther = list()
for key, value in dictOfElems.items():
    listHash.append(key)
    listOther.append(value)

In [16]:
listHash = np.array(listHash)
listOther = np.array(listOther)
bkg_train_freq = listOther[:,0]

In [18]:
bkg_train_freq_ind = listOther[:,1]

In [31]:
len(bkg_train_freq_ind[1])

81513

In [32]:
bkg_train_freq[1]

81513

In [33]:
from sklearn.cluster import KMeans

In [34]:
bkg_train_freq_ind[0]

[0,
 132,
 194,
 247,
 316,
 320,
 337,
 358,
 392,
 411,
 414,
 415,
 458,
 497,
 518,
 541,
 547,
 580,
 644,
 686,
 732,
 739,
 744,
 903,
 914,
 1100,
 1105,
 1128,
 1144,
 1174,
 1254,
 1276,
 1281,
 1297,
 1349,
 1440,
 1480,
 1577,
 1600,
 1631,
 1633,
 1713,
 1782,
 1908,
 2013,
 2308,
 2351,
 2401,
 2467,
 2592,
 2599,
 2624,
 2646,
 2693,
 2702,
 2782,
 2792,
 2800,
 2804,
 2805,
 2807,
 2832,
 2878,
 2946,
 2948,
 2962,
 3001,
 3017,
 3029,
 3076,
 3078,
 3099,
 3167,
 3198,
 3213,
 3225,
 3308,
 3318,
 3409,
 3450,
 3473,
 3520,
 3562,
 3606,
 3607,
 3734,
 3849,
 3873,
 4020,
 4082,
 4178,
 4225,
 4249,
 4321,
 4328,
 4368,
 4382,
 4407,
 4431,
 4438,
 4452,
 4510,
 4528,
 4543,
 4586,
 4611,
 4640,
 4706,
 4721,
 4742,
 4797,
 4798,
 4803,
 4822,
 4889,
 4954,
 5205,
 5233,
 5263,
 5272,
 5397,
 5418,
 5463,
 5522,
 5566,
 5596,
 5699,
 5718,
 5785,
 5826,
 5958,
 5979,
 6032,
 6074,
 6108,
 6147,
 6173,
 6185,
 6191,
 6197,
 6248,
 6249,
 6265,
 6278,
 6343,
 6369,
 6593

In [37]:
encodedBuck = encoded_x_train[bkg_train_freq_ind[0]]

In [38]:
encodedBuck.shape

(11328, 10)

In [36]:
encoded_x_train[0]

array([-0.00900374, -0.0046296 , -0.01104837,  0.83119416,  0.01010827,
        0.00225644,  0.06753303,  0.00488269, -0.0178441 ,  0.43490672],
      dtype=float32)

In [39]:
Kmean = KMeans(n_clusters=1)
Kmean.fit(encodedBuck)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=1, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [40]:
Kmean.cluster_centers_

array([[ 0.00754179,  0.00141694, -0.04193572,  0.59201616,  0.01767498,
         0.00063935,  0.03689159,  0.02121503,  0.0038847 ,  0.27872616]],
      dtype=float32)

In [19]:
bkg_train_freq = list(bkg_train_freq)

In [20]:
tableSize= len(listHash)
sample_size, vec_size = encoded_x_test.shape
hash_truth = list()
bkgCount = list()
count = 0
for i in range(sample_size):
    bools = ''
    a = 1
    for j in range(hash_size):
        bool = (np.dot(encoded_x_test[i], projections[j].T) > 0).astype('int')
        #print(bool)
        bools += str(bool)
        #print(bools)
    for k in range(tableSize):
        if bools == listHash[k]:
            bkgCount.append(k)
            a=0
            break
    if(a==1):
        count+=1
    hash_truth.append(a)
print(count)

138


In [21]:
bkg = Counter(bkgCount)
bkga=sorted(bkg.items())
bkg_test_freq = [x[1] for x in bkga]

In [22]:
filename_1 = '/lcg/storage13/atlas/gupta/bkg_train_freq'+'str(level)'+'str(hash_size)'+'.h5'
h5f_train_freq = h5py.File(filename_1, 'w')
h5f_train_freq.create_dataset('table', data=bkg_train_freq, compression="gzip")
h5f_train_freq.close()

In [23]:
filename_2 = '/lcg/storage13/atlas/gupta/bkg_test_freq'+'str(level)'+'str(hash_size)'+'.h5'
h5f_test_freq = h5py.File(filename_2, 'w')
h5f_test_freq.create_dataset('table', data=bkg_test_freq, compression="gzip")
h5f_test_freq.close()

In [24]:
filename_3 = '/lcg/storage13/atlas/gupta/hash_truth'+'str(level)'+'str(hash_size)'+'.h5'
h5f_hash_truth = h5py.File(filename_3, 'w')
h5f_hash_truth.create_dataset('table', data=hash_truth, compression="gzip")
h5f_hash_truth.close()

In [25]:
f_encoded_train.close()
f_encoded_test.close()